In [1]:
import logging
import modelseedpy
import cobrakbase
import cobra
import json
logger = logging.getLogger(__name__)

cobrakbase 0.2.7


In [2]:
kbase = cobrakbase.KBaseAPI()
ws = 'filipeliu:narrative_1623733614496'

In [3]:
info = kbase.get_object_info_from_ref('78503/2948/1')

In [13]:
genome_json = kbase.get_object(info.id, info.workspace_id)

In [14]:
from cobrakbase.core import KBaseGenome
import copy
genome = KBaseGenome(copy.deepcopy(genome_json), info)

In [15]:
genome.data_keys

{'assembly_ref': str,
 'cdss': list,
 'dna_size': int,
 'domain': str,
 'external_source_origination_date': str,
 'feature_counts': dict,
 'features': list,
 'gc_content': float,
 'genbank_handle_ref': str,
 'genetic_code': int,
 'genome_tiers': list,
 'id': str,
 'md5': str,
 'molecule_type': str,
 'mrnas': list,
 'non_coding_features': list,
 'notes': str,
 'num_contigs': int,
 'ontologies_present': dict,
 'ontology_events': list,
 'original_source_file_name': str,
 'publications': list,
 'reference_annotation': int,
 'release': str,
 'scientific_name': str,
 'source': str,
 'source_id': str,
 'taxon_ref': str,
 'taxonomy': str,
 'type': str,
 'warnings': list}

In [16]:
genome_json_back = genome.get_data()
genome_json_back.keys() == genome_json.keys()

True

In [27]:
data['ontology_events']

[{'id': 'SSO',
  'method': 'annotate_genomes',
  'method_version': '1',
  'ontology_ref': '17159/4/1',
  'timestamp': '2018-04-23T06:59:27'},
 {'id': 'SSO',
  'method': 'annotate_genome',
  'method_version': '1.8.1',
  'ontology_ref': '17159/4/1',
  'timestamp': '2020-12-01T14:40:29'},
 {'id': 'SSO',
  'method': 'annotate_genome',
  'method_version': '1.8.1',
  'ontology_ref': '17159/4/1',
  'timestamp': '2020-12-01T14:40:29'}]

In [26]:
from cobra.core.dictlist import DictList
import copy
class AttrDict(dict):
    """
    Base object to use for subobjects in KBase objects
    """
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
data = copy.deepcopy(genome_json)
for field in data.keys():
    if isinstance(data[field], list) and len(data[field]) > 0:
        logger.warning("object field: %s", field)
        if isinstance(data[field][0], dict) and "id" in data[field][0]:
            data[field] = DictList(map(lambda x: AttrDict(x), data[field]))
            pass

ValueError: id 'SSO' at index 1 is non-unique. Is it present twice?

In [6]:
from cobrakbase.core.kbaseobject import KBaseObject

In [7]:
base = KBaseObject({}, info)

class KBaseGenome2(KBaseObject):
    def __init__(self, data=None, info=None, args=None, kbase_type=None):
        KBaseObject.__init__(self, data, info, args, kbase_type, ['ontology_events'])
        
ggg = KBaseGenome2({'ontology_events': [{'id':'SSO'}]}, info)

In [9]:
ggg.exclude_dict

{'ontology_events'}

In [3]:
#genome = kbase.get_from_ws('ecoli_genome', ws)
from cobrakbase.core.kbasegenomesgenome import KBaseGenome
with open('ecoli_genome.json', 'r') as fh:
    genome_json = json.load(fh)
    genome = KBaseGenome(genome_json, cobrakbase.kbase_object_info.KBaseObjectInfo('1-1'))

In [4]:
with open('ecoli_genome_rast_annotation.json', 'r') as fh:
    ecoli_genome_rast_annotation = json.load(fh)
for f in genome.features:
    if f.id in ecoli_genome_rast_annotation:
        for function in ecoli_genome_rast_annotation[f.id]:
            f.add_ontology_term('RAST', function)

In [5]:
template = kbase.get_from_ws('12998/10/6')

In [6]:
model_modelseed = modelseedpy.MSBuilder.build_metabolic_model('ecoli', genome, template, None, '0', True, False)

In [8]:
model_modelseed

Name,ecoli
Memory address,0x07f85df00ac70
Number of metabolites,1448
Number of reactions,1738
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"c0, e0"


In [7]:
kbase.save_object('ecoli_modelseedpy_direct', 'filipeliu:narrative_1623733614496', 'KBaseFBA.FBAModel', model_modelseed)

[[17,
  'ecoli_modelseedpy_direct',
  'KBaseFBA.FBAModel-12.0',
  '2021-06-18T06:16:38+0000',
  14,
  'filipeliu',
  92991,
  'filipeliu:narrative_1623733614496',
  '08231a4979927f14ae3f889b74b1e9f4',
  1570596,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'ecoli',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Source': 'ModelSEEDpy',
   'Number compounds': '1448',
   'Number reactions': '1564',
   'Name': 'ecoli'}]]

In [8]:
rast = modelseedpy.RastClient()

In [9]:
rast.annotate_genome(genome)

[{'parameters': ['-a',
   '-g',
   200,
   '-m',
   5,
   '-d',
   '/opt/patric-common/data/kmer_metadata_v2',
   '-u',
   'http://pear.mcs.anl.gov:6100/query'],
  'hostname': 'pear',
  'tool_name': 'kmer_search',
  'id': '40F7D9AC-CF98-11EB-A0A4-763ABEF382BD',
  'execution_time': 1623953839.68946},
 {'tool_name': 'KmerAnnotationByFigfam',
  'parameters': ['annotate_hypothetical_only=1',
   'dataset_name=Release70',
   'kmer_size=8'],
  'hostname': 'pear',
  'id': '410F6400-CF98-11EB-A0A4-763ABEF382BD',
  'execution_time': 1623953839.84372},
 {'tool_name': 'annotate_proteins_similarity',
  'execute_time': 1623953840.21613,
  'hostname': 'pear',
  'parameters': [],
  'id': '4148369A-CF98-11EB-9BC7-0F0BBDF382BD'}]

In [8]:
model_kbase_json = kbase.get_object('ecoli_model', ws)

In [9]:
model_kbase = kbase.get_from_ws('ecoli_model', ws)

In [10]:
to_json = model_kbase._to_json()

In [11]:
to_json == model_kbase_json

True

In [12]:
model_kbase.info.type

'KBaseFBA.FBAModel'

In [13]:
model_kbase_json['modelcompartments']

[{'compartmentIndex': 0,
  'compartment_ref': '~/template/compartments/id/c',
  'id': 'c0',
  'label': 'Cytosol_0',
  'pH': 7,
  'potential': 0},
 {'compartmentIndex': 0,
  'compartment_ref': '~/template/compartments/id/e',
  'id': 'e0',
  'label': 'Extracellular_0',
  'pH': 7,
  'potential': 0}]

In [14]:
model_modelseed_json['modelcompartments']

NameError: name 'model_modelseed_json' is not defined

In [ ]:
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
model_modelseed_json, model_modelseed_fbamodel = CobraModelConverter(model_modelseed, genome, template).build()

In [18]:
kbase.save_object('ecoli_modelseedpy', 'filipeliu:narrative_1623733614496', model_modelseed_fbamodel.info.type, model_modelseed_json)

[[11,
  'ecoli_modelseedpy',
  'KBaseFBA.FBAModel-12.0',
  '2021-06-16T08:30:30+0000',
  3,
  'filipeliu',
  92991,
  'filipeliu:narrative_1623733614496',
  'e96e8d34e1deba910dac1e2aa4e99976',
  1567072,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'COBRA KBase',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Source': 'COBRA KBase',
   'Number compounds': '1446',
   'Number reactions': '1560',
   'Name': 'ecoli'}]]

In [105]:
for r in model_modelseed.reactions:
    if r.id == 'rxn00646_c0':
        proteins = []
        model_reaction_data = {
                'id': r.id,
                'name': r.name,
                'direction': '>',
                'aliases': [],
                'coverage': 3,
                'dblinks': {},
                'edits': {}, 'gapfill_data': {},
                'gene_count': 3,
                'maxforflux': 1000000,
                'maxrevflux': 1000000,
                'modelReactionProteins': proteins,
                'modelReactionReagents': [],
                'modelcompartment_ref': '~/modelcompartments/id/c0',
                'numerical_attributes': {}, 'string_attributes': {},
                'probability': 1,
                'protons': 0,
                'reaction_ref': '~/template/reactions/id/' + r.id[:-1],
            }
        reaction = ModelReaction(model_reaction_data)
        reaction.add_metabolites(r.metabolites)
        print(r)
        print(reaction)
        break

rxn00646_c0: cpd00002_c0 + cpd00023_c0 + cpd00084_c0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd00506_c0
rxn00646_c0: cpd00002_c0 + cpd00023_c0 + cpd00084_c0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd00506_c0


In [18]:
r

Reaction identifier,SK_cpd15302_c0
Name,Sink for glycogen(n-1)
Memory address,0x07f9cadb25410
Stoichiometry,cpd15302_c0 --> glycogen(n-1) -->
GPR,
Lower bound,0
Upper bound,1000


In [24]:
model_modelseed.compartments

{'c0': '', 'e0': ''}

In [120]:
def tt_to_json(r):
    rev, max_flux, min_flux = r._get_rev_and_max_min_flux()
    model_reaction_reagents = []
    model_reaction_proteins = []
    s = r.metabolites
    for m in s:
        model_reaction_reagents.append({
            'coefficient': s[m],
            'modelcompound_ref': '~/modelcompounds/id/' + m.id
        })
    return {
        'id': r.id,
        'name': r.name,
        'direction': rev,
        'maxforflux': max_flux,
        'maxrevflux': min_flux,
        'aliases': [],
        'dblinks': {},
        'edits': {},
        'gapfill_data': {},
        'modelReactionProteins': model_reaction_proteins,
        'modelReactionReagents': model_reaction_reagents,
        'modelcompartment_ref': '~/modelcompartments/id/' + r.compartment,
        'numerical_attributes': {},
        'probability': 0,
        'protons': 0,
        'reaction_ref': '~/template/reactions/id/' + r.id[:-1],
        'string_attributes': {}
    }
tt_to_json(reaction)

{'id': 'rxn00646_c0',
 'name': 'L-glutamate:L-cysteine gamma-ligase (ADP-forming)_c0',
 'direction': '=',
 'maxforflux': 1000,
 'maxrevflux': 0.0,
 'aliases': [],
 'dblinks': {},
 'edits': {},
 'gapfill_data': {},
 'modelReactionProteins': [],
 'modelReactionReagents': [{'coefficient': -1,
   'modelcompound_ref': '~/modelcompounds/id/cpd00023_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00084_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00002_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00008_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00009_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00506_c0'}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn00646_c',
 'stri

In [61]:
debug['template_refs']

['']

In [70]:
debug['modelcompartments']

[{'compartmentIndex': 0,
  'compartment_ref': '~/template/compartments/id/c',
  'id': 'c0',
  'label': '',
  'pH': 7,
  'potential': 0}]

In [20]:
import cobra.test
model = cobra.test.create_test_model("textbook")

In [23]:
r = model.reactions.get_by_id('ICDHyr')

In [26]:
r.gene_reaction_rule

'b1136'

In [13]:
debug = kbase.get_object('ecoli_modelseedpy_direct', ws)

In [16]:
for k in debug:
    v = debug[k]
    if type(v) == list:
        print(k, '\tList:', len(v))
    else:
        print(k,'\t[', v, ']')

__VERSION__ 	[ 1 ]
attributes 	[ {'auxotrophy': {}, 'fbas': {}, 'gene_count': 0, 'pathways': {}} ]
biomasses 	List: 1
contig_coverages 	[ {} ]
delete_biomasses 	[ {} ]
deleted_reactions 	[ {} ]
gapfilledcandidates 	List: 0
gapfillings 	List: 0
gapgens 	List: 0
id 	[ ecoli_modelseedpy_direct ]
loops 	List: 0
model_edits 	List: 0
modelcompartments 	List: 2
modelcompounds 	List: 1446
modelreactions 	List: 1560
name 	[ ecoli ]
other_genome_refs 	List: 0
quantopts 	List: 0
source 	[ ModelSEEDpy ]
source_id 	[ ecoli ]
template_ref 	[ 12998/10/6 ]
template_refs 	List: 1
type 	[ GenomeScale ]


In [15]:
debug['template_ref'] = str(template.info)
debug['template_refs'] = [str(template.info)]

In [17]:
kbase.save_object('ecoli_modelseedpy_direct', ws, model_kbase.info.type, debug)

[[17,
  'ecoli_modelseedpy_direct',
  'KBaseFBA.FBAModel-12.0',
  '2021-06-16T10:04:26+0000',
  8,
  'filipeliu',
  92991,
  'filipeliu:narrative_1623733614496',
  '237e105b19a24b2496f30213c8483664',
  1569437,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'ecoli',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Source': 'ModelSEEDpy',
   'Number compounds': '1446',
   'Number reactions': '1560',
   'Name': 'ecoli'}]]

In [97]:
reaction = model_modelseed.reactions[6]

In [ ]:
model = kbase.get_from_ws

In [23]:
from cobra.core import Model

In [15]:
model1 = kbase.get_object('ecoli_modelseedpy_direct', ws)
model2 = kbase.get_object('ecoli_kbase_classic', ws)

In [16]:
compounds1 = dict(map(lambda x: (x['id'], x), model1['modelcompounds']))
compounds2 = dict(map(lambda x: (x['id'], x), model2['modelcompounds']))
reactions1 = dict(map(lambda x: (x['id'], x), model1['modelreactions']))
reactions2 = dict(map(lambda x: (x['id'], x), model2['modelreactions']))

In [17]:
def compare_compound(m1, m2):
    assert m1['id'] == m2['id']
    #assert m1['name'] == m2['name']
    assert m1['compound_ref'] == m2['compound_ref']
    assert m1['modelcompartment_ref'] == m2['modelcompartment_ref']
    assert m1['formula'] == m2['formula']

In [18]:

for m1 in compounds1:
    if m1 in compounds2:
        compare_compound(compounds1[m1], compounds2[m1])
    else:
        print(m1)
for m2 in compounds2:
    if m2 not in compounds1:
        print(m2, compounds2[m2]['name'])

cpd00215_e0 Pyridoxal_e0
cpd00218_e0 Niacin_e0
cpd00220_e0 Riboflavin_e0
cpd00017_e0 S-Adenosyl-L-methionine_e0
cpd00393_e0 Folate_e0
cpd00084_e0 L-Cysteine_e0
cpd00104_e0 BIOT_e0


In [31]:
from cobrakbase.core.kbasefba.fbamodel_from_cobra import CobraModelConverter
model_modelseed_json, model_modelseed_fbamodel = CobraModelConverter(model_modelseed, None, None).build()

In [35]:
model_modelseed_fbamodel.reactions.rxn08502_e0._to_json()

{'id': 'rxn08502_e0',
 'name': 'enterobactin Fe(III) binding (spontaneous)_c0',
 'direction': '=',
 'maxforflux': 1000,
 'maxrevflux': 0.0,
 'aliases': [],
 'dblinks': {},
 'edits': {},
 'gapfill_data': {},
 'modelReactionProteins': [],
 'modelReactionReagents': [{'modelcompound_ref': '~/modelcompounds/id/cpd03726_e0',
   'coefficient': 1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd03453_e0',
   'coefficient': -1.0},
  {'modelcompound_ref': '~/modelcompounds/id/cpd10516_e0',
   'coefficient': -1.0}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn08502_e',
 'string_attributes': {}}

'k0'

In [37]:
r = model_modelseed.reactions.rxn13783_c0

In [38]:
r

Reaction identifier,rxn13783_c0
Name,DNA replication_c0
Memory address,0x07fb9fa7d9710
Stoichiometry,--> cpd17042_c0 --> DNA replication
GPR,(b4059) or (b0060) or (b2231) or (b3699) or (b1274) or (b0215) or (b4259) or (b3066) or (b3935) o...
Lower bound,0
Upper bound,1000


In [24]:
reactions2[r1]

{'aliases': [],
 'coverage': 0,
 'dblinks': {},
 'direction': '>',
 'edits': {},
 'gapfill_data': {},
 'gene_count': 0,
 'id': 'rxn08502_e0',
 'maxforflux': 1000000,
 'maxrevflux': 1000000,
 'modelReactionProteins': [{'complex_ref': '',
   'modelReactionProteinSubunits': [],
   'note': 'spontaneous',
   'source': ''}],
 'modelReactionReagents': [{'coefficient': -1,
   'modelcompound_ref': '~/modelcompounds/id/cpd03453_e0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd10516_e0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd03726_e0'}],
 'modelcompartment_ref': '~/modelcompartments/id/e0',
 'name': 'enterobactin Fe(III) binding (spontaneous)_e0',
 'numerical_attributes': {},
 'probability': 1,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn08502_e',
 'string_attributes': {}}

In [19]:
def assert_reaction(r1, r2):
    s1 = dict(map(lambda x: (x['modelcompound_ref'], x['coefficient']), r1['modelReactionReagents']))
    s2 = dict(map(lambda x: (x['modelcompound_ref'], x['coefficient']), r2['modelReactionReagents']))
    
    assert r1['id'] == r2['id']
    assert s1 == s2
    assert r1['direction'] == r2['direction']
    assert r1['reaction_ref'] == r2['reaction_ref']
    assert r1['modelcompartment_ref'] == r2['modelcompartment_ref']
    #print(r1['probability'])
    #print(r2['probability'])
    #print(r1['modelReactionProteins'])
    #print(r2['modelReactionProteins'])
    #print(r2)
    #print(r1)
    #print(r1['name'])
    #print(r2['name'])
    #assert r1['name'] == r2['name']
    

assert_reaction(reactions1[r1], reactions2[r1])

NameError: name 'r1' is not defined

In [26]:
trxn = template.reactions.get_by_id('rxn05310_c')

In [31]:
#trxn

In [21]:
reactions2['rxn05310_c0']

{'aliases': [],
 'dblinks': {},
 'direction': '=',
 'edits': {},
 'gapfill_data': {},
 'id': 'rxn05310_c0',
 'maxforflux': 1000000,
 'maxrevflux': 1000000,
 'modelReactionProteins': [],
 'modelReactionReagents': [{'coefficient': 1,
   'modelcompound_ref': '~/modelcompounds/id/cpd00218_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00218_e0'}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'name': 'Nicotinate transport (Plasma membrane)_c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn05310_c',
 'string_attributes': {}}

In [20]:
for r1 in reactions1:
    if r1 in reactions2:
        assert_reaction(reactions1[r1], reactions2[r1])
    else:
        print(r1, reactions1[r1]['name'])
for r2 in reactions2:
    if r2 not in reactions1:
        print(r2, reactions2[r2]['name'])

rxn03038_c0 R04376_c0
rxn04603_c0 R06788_c0
rxn05299_c0 TRANS-RXNBWI-115637.ce.maizeexp.HIS_c0
rxn05244_c0 TRANS-RXNBWI-115637.ce.maizeexp.ILE_c0
rxn12666_c0 Pyridoxal transport_c0
rxn05308_c0 Pantothenate reversible transport via proton symport_c0
rxn05305_c0 L-lysine transport out via proton antiport reversible_c0
rxn09672_c0 TRANS-RXNBWI-115637.ce.maizeexp.MET_c0
rxn00068_c0 Fe2+:NAD+ oxidoreductase_c0
rxn05310_c0 Nicotinate transport (Plasma membrane)_c0
rxn05243_c0 TRANS-RXNBWI-115637.ce.maizeexp.LEU_c0
rxn05645_c0 riboflavin transport in via proton symport_c0
rxn09678_c0 TRANS-RXNBWI-115637.ce.maizeexp.GLN_c0
rxn09696_c0 S-adenosylmethionine permease SAM3_c0
rxn05255_c0 folate transport via proton simport_c0
rxn05303_c0 TRANS-RXNBWI-115637.ce.maizeexp.ARG_c0
rxn05669_c0 TRANS-RXNBWI-115637.ce.maizeexp.VAL_c0
rxn09657_c0 Thiamine transporter_c0
rxn09690_c0 TRANS-RXNBWI-115637.ce.maizeexp.CYS_c0
rxn09693_c0 Biotin uptake_c0


In [24]:
len(model1['modelreactions'])

1561

In [25]:
len(model2['modelreactions'])

1580

In [26]:
len(model1['modelcompounds'])

1448

In [27]:
len(model2['modelcompounds'])

1455

In [42]:
genome = kbase.get_from_ws('ecoli_genome.rast', ws)

In [43]:
ecoli_genome_rast_annotation = {}
for feature in genome.features:
    ecoli_genome_rast_annotation[feature.id] = list(feature.functions)
with open('ecoli_genome_rast_annotation.json', 'w') as fh:
    fh.write(json.dumps(ecoli_genome_rast_annotation))

In [39]:
import logging
logger = logging.getLogger(__name__)

In [22]:
aa = "rxn11111_c"
aa[-1]

'c'

In [32]:
from modelseedpy.core.msbuilder import aaaa

In [36]:
from modelseedpy.core.msgenome import normalize_role
def aaaa(genome):
    search_name_to_genes = {}
    search_name_to_orginal = {}
    for f in genome.features:
        if 'RAST' in f.ontology_terms:
            functions = f.ontology_terms['RAST']
            for function in functions:
                f_norm = normalize_role(function)
                if f_norm not in search_name_to_genes:
                    search_name_to_genes[f_norm] = set()
                    search_name_to_orginal[f_norm] = set()
                search_name_to_orginal[f_norm].add(function)
                search_name_to_genes[f_norm].add(f.id)
    return search_name_to_genes, search_name_to_orginal




In [37]:
search_name_to_genes, search_name_to_orginal = aaaa(genome)

In [44]:
rxn_roles = {}
roles = dict(map(lambda x: (x['id'], x), template.roles))
for r in template.reactions:
    rxn_roles[r.id] = set()

In [103]:
template_reaction = template.reactions.get_by_id('rxn00148_c')
template_reaction['templatecomplex_refs']
print(template_reaction)

rxn00148_c: cpd00002_c [c] + cpd00020_c [c] --> cpd00008_c [c] + cpd00061_c [c] + cpd00067_c [c]


In [66]:
?cpx

Type:        AttrDict
String form: {'complexroles': [{'optional_role': 0, 'templaterole_ref': '~/roles/id/ftr01381', 'triggering': 1 <...> nfidence': 1, 'id': 'cpx01814', 'name': 'mscpx.1814', 'reference': 'null', 'source': 'ModelSEED'}
Length:      6
File:        /usr/local/lib/python3.8/site-packages/cobrakbase/core/kbaseobject.py
Docstring:   Base object to use for subobjects in KBase objects


In [81]:
# rxn_id -> cpx_id -> role_id -> [normalized_function: str, triggering: bool, optional_role: bool, match_genes: set]
match_complex = {
    'rxn00154_c': {
        'cpx01524': {
            'ftr01383': [normalize_role('Dihydrolipoamide dehydrogenase of pyruvate dehydrogenase complex (EC 1.8.1.4)'), True, False, set()]
        }
    }
}
match_complex

{'rxn00154_c': {'cpx01524': {'ftr01383': ['dihydrolipoamidedehydrogenaseofpyruvatedehydrogenasecomplexec1814',
    True,
    False,
    set()]}}}

In [102]:
for k in ['b1854', 'b1676', 'b0723', 'b0724']:
    print(k, genome.features.get_by_id(k).ontology_terms)

b1854 {'RAST': ['Pyruvate kinase (EC 2.7.1.40)']}
b1676 {'RAST': ['Pyruvate kinase (EC 2.7.1.40)']}
b0723 {'RAST': ['Succinate dehydrogenase flavoprotein subunit (EC 1.3.5.1)']}
b0724 {'RAST': ['Succinate dehydrogenase iron-sulfur protein (EC 1.3.5.1)']}


In [168]:
mrp0 = {
    'complex_ref': '~/template/complexes/name/cpx00108',
    'modelReactionProteinSubunits': [
        {'feature_refs': ['~/genome/features/id/b4151'], 'note': '', 'optionalSubunit': 0, 'role': 'ftr07617', 'triggering': 1},
        {'feature_refs': ['~/genome/features/id/b0722'], 'note': '', 'optionalSubunit': 0, 'role': 'ftr09157', 'triggering': 1},
        {'feature_refs': ['~/genome/features/id/b0721'], 'note': '', 'optionalSubunit': 0, 'role': 'ftr09156', 'triggering': 1},
        {'feature_refs': ['~/genome/features/id/b4152'], 'note': '', 'optionalSubunit': 0, 'role': 'ftr07616', 'triggering': 1}
    ],
   'note': '',
   'source': ''}
mrps = [
    mrp0
]
mrps

[{'complex_ref': '~/template/complexes/name/cpx00108',
  'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b4151'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07617',
    'triggering': 1},
   {'feature_refs': ['~/genome/features/id/b0722'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr09157',
    'triggering': 1},
   {'feature_refs': ['~/genome/features/id/b0721'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr09156',
    'triggering': 1},
   {'feature_refs': ['~/genome/features/id/b4152'],
    'note': '',
    'optionalSubunit': 0,
    'role': 'ftr07616',
    'triggering': 1}],
  'note': '',
  'source': ''}]

In [166]:
template_reaction = template.reactions.get_by_id('rxn09272_c')
print(template_reaction['templatecomplex_refs'])
template_reaction_complexes = get_template_reaction_complexes(template_reaction)
list(filter(lambda x: x['id'] == 'rxn09272_c0', model_kbase_json['modelreactions']))[0]['modelReactionProteins'], template_reaction_complexes

['~/complexes/id/cpx00107', '~/complexes/id/cpx00108', '~/complexes/id/cpx00109', '~/complexes/id/tcpx00642', '~/complexes/id/tcpx00643', '~/complexes/id/tcpx00644', '~/complexes/id/tcpx00645', '~/complexes/id/tcpx00646', '~/complexes/id/cpx00272', '~/complexes/id/tcpx00647']


([{'complex_ref': '~/template/complexes/name/cpx00108',
   'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b4151'],
     'note': '',
     'optionalSubunit': 0,
     'role': 'ftr07617',
     'triggering': 1},
    {'feature_refs': ['~/genome/features/id/b0722'],
     'note': '',
     'optionalSubunit': 0,
     'role': 'ftr09157',
     'triggering': 1},
    {'feature_refs': ['~/genome/features/id/b0721'],
     'note': '',
     'optionalSubunit': 0,
     'role': 'ftr09156',
     'triggering': 1},
    {'feature_refs': ['~/genome/features/id/b4152'],
     'note': '',
     'optionalSubunit': 0,
     'role': 'ftr07616',
     'triggering': 1}],
   'note': '',
   'source': ''},
  {'complex_ref': '~/template/complexes/name/tcpx00642',
   'modelReactionProteinSubunits': [{'feature_refs': ['~/genome/features/id/b0724'],
     'note': '',
     'optionalSubunit': 0,
     'role': 'tftr00456',
     'triggering': 1}],
   'note': '',
   'source': ''},
  {'complex_ref': '~/template

In [284]:
#match_complex['rxn09272_c']

In [189]:
import logging
logger = logging.getLogger(__name__)

In [278]:
match_complex = {
    'rxn09272_c': {
        'sdh': {
            'sdhC': [normalize_role('Succinate dehydrogenase cytochrome b-556 subunit'), True, False, set()],
            'sdhD': [normalize_role('Succinate dehydrogenase hydrophobic membrane anchor protein'), True, False, set()],
            'sdhA': [normalize_role('Succinate dehydrogenase flavoprotein subunit (EC 1.3.5.1)'), True, False, set()],
            'sdhB': [normalize_role('Succinate dehydrogenase iron-sulfur protein (EC 1.3.5.1)'), True, False, set()],
            'sdhZ': [normalize_role('Succinate dehydrogenase iron-sulfur protein (zzzzzzzzzz)'), True, False, set()],
        }
    },
    'rxn00148_c': {
        'pyk': {
            'pykAF': [normalize_role('Pyruvate kinase (EC 2.7.1.40)'), True, False, set()]
        }
    },
    'not_found': {
        'xxx': {
            'xxxABC': [normalize_role('this role should not be mapped'), True, False, set()],
        }
    },
}


def build_reaction_complex_gpr_sets(match_complex, search_name_to_genes, search_name_to_orginal, allow_incomplete_complexes=True):
    metabolic_reactions_2 = {}
    for rxn_id in match_complex:
        complexes = {}
        for cpx_id in match_complex[rxn_id]:
            complete = True
            roles = set()
            role_genes = {}
            for role_id in match_complex[rxn_id][cpx_id]:
                t = match_complex[rxn_id][cpx_id][role_id]
                complete &= len(t[3]) > 0 or not t[1] or t[2] # true if has genes or is not triggering or is optional
                #print(t[3])
                if len(t[3]) > 0:
                    roles.add(role_id)
                    role_genes[role_id] = t[3]
                #print(t)
            # it is never complete if has no genes, only needed if assuming a complex can have all roles be either non triggering or optional
            logger.warning('%s maps to %s and complete: %s', cpx_id, roles, complete)
            if len(roles) > 0 and (allow_incomplete_complexes or complete):
                if rxn_id not in metabolic_reactions_2:
                    metabolic_reactions_2[rxn_id] = {}
                ll = []
                for role_id in role_genes:
                    role_gene_set = []
                    for gene_id in role_genes[role_id]:
                        role_gene_set.append({gene_id: role_id})
                    ll.append(role_gene_set)
                print(role_genes)
                metabolic_reactions_2[rxn_id][cpx_id] = list(map(lambda x: dict(map(lambda o: list(o.items())[0], x)), itertools.product(*ll)))
    return metabolic_reactions_2
                
map_genes(match_complex, search_name_to_genes, search_name_to_orginal)
mr = build_reaction_complex_gpr_sets(match_complex, search_name_to_genes, search_name_to_orginal, True)
mr

{'sdhC': {'b0721'}, 'sdhD': {'b0722'}, 'sdhA': {'b0723'}, 'sdhB': {'b0724'}}
{'pykAF': {'b1854', 'b1676'}}


{'rxn09272_c': {'sdh': [{'b0721': 'sdhC',
    'b0722': 'sdhD',
    'b0723': 'sdhA',
    'b0724': 'sdhB'}]},
 'rxn00148_c': {'pyk': [{'b1854': 'pykAF'}, {'b1676': 'pykAF'}]}}

In [ ]:
match_complex = {}
for template_reaction in template.reactions:
    template_reaction_complexes = get_template_reaction_complexes(template_reaction)

    if len(template_reaction_complexes) > 0:
        match_complex[template_reaction.id] = template_reaction_complexes

In [292]:
def map_genes(match_complex, search_name_to_genes, search_name_to_orginal):
    metabolic_reactions_2 = {}
    for rxn_id in match_complex:
        for cpx_id in match_complex[rxn_id]:
            for role_id in match_complex[rxn_id][cpx_id]:
                t = match_complex[rxn_id][cpx_id][role_id]
                if t[0] in search_name_to_genes:
                    t[3] |= search_name_to_genes[t[0]]
                    

map_genes(match_complex, search_name_to_genes, search_name_to_orginal)

In [310]:



gpr_set = get_gpr_from_template_reaction(template.reactions.get_by_id('rxn00148_c'))
print('gpr_set', gpr_set)
build_gpr(gpr_set)

gpr_set {'cpx01664': [{'b1854': 'ftr00229'}, {'b1676': 'ftr00229'}]}


'(b1854) or (b1676)'

In [313]:
for template_reaction in template.reactions:
    gpr_set = get_gpr_from_template_reaction(template_reaction)
    if gpr_set:
        print(template_reaction.id, build_gpr(gpr_set))

rxn02201_c (b3177) or (b0142)
rxn00351_c (b2947)
rxn00836_c (b0125)
rxn05318_c (b2964)
rxn01094_c (b0596)
rxn08709_c (b3625)
rxn00423_c (b3607)
rxn00364_c (b0910)
rxn05561_c (b3528) or (b4138) or (b0621) or (b4123)
rxn03408_c (b0090)
rxn02177_c (b0595) or (b0595)
rxn05250_c (b1805)
rxn00646_c (b2688)
rxn05440_c (b1288)
rxn00935_c (b2210)
rxn01673_c (b2518)
rxn00247_c (b3403)
rxn05625_c (b1469) or (b1223)
rxn02342_c (b0114)
rxn05156_c (b2310) or (b2309) or (b2306) or (b2307) or (b2308)
rxn01452_c (b1395)
rxn00642_c (b2053)
rxn00022_c (b0403)
rxn02000_c (b2038)
rxn00322_c (b4131) or (b0186)
rxn05457_c (b1092) or (b1028)
rxn00107_c (b2518)
rxn13783_c (b0060) or (b0215) or (b4372) or (b4059 and b3702 and b1274 and b2231 and b0184 and b1099 and b0470 and b0640 and b3699 and b3935 and b3701) or (b3066) or (b1842) or (b4259)
rxn08734_c (b2721 and b2720 and b2722 and b0733 and b2723 and b2724 and b2719) or (b0978 and b2721 and b2720 and b2722 and b2723 and b2724 and b2719)
rxn08934_c (b0403)
r

In [318]:
build_gpr({})

''

In [254]:
{'cpx': [[{'c': 'pykAF'}, {'a': 'pykB'}, {'b': 'pykC'}], [{"c'": 'pykAF'}, {'a': 'pykB'}, {'b': 'pykC'}]]}

{'cpx': [[{'c': 'pykAF'}, {'a': 'pykB'}, {'b': 'pykC'}],
  [{"c'": 'pykAF'}, {'a': 'pykB'}, {'b': 'pykC'}]]}

In [201]:
metabolic_reactions_2_expect = {
    'rxn09272_c': {
        'sdh': [{'b0721': 'sdhC', 'b0722': 'sdhD', 'b0723': 'sdhA', 'b0724': 'sdhB'}]
    },
    'rxn00148_c': {
        'pyk': [{'b1676': 'pykAF'}, {'b1854': 'pykAF'}]
    }
}
metabolic_reactions_2_expect

{'rxn09272_c': {'sdh': [{'b0721': 'sdhC',
    'b0722': 'sdhD',
    'b0723': 'sdhA',
    'b0724': 'sdhB'}]},
 'rxn00148_c': {'pyk': [{'b1676': 'pykAF'}, {'b1854': 'pykAF'}]}}

In [306]:
def build_gpr(cpx_gene_role):
    # save cpx_id and role_id in reaction annotation for KBase object
    gpr_or_ll = []
    for cpx_id in cpx_gene_role:
        #print(cpx_id)
        for complex_set in cpx_gene_role[cpx_id]:
            #print(complex_set)
            gpr_or_ll.append("({})".format(' and '.join(set(complex_set))))
    
    return ' or '.join(gpr_or_ll)
print(build_gpr({'sdh': [{'b0721': 'sdhC', 'b0722': 'sdhD', 'b0723': 'sdhA', 'b0724': 'sdhB'}]}))
print(build_gpr({'cpx1': [{'g1': 'role1', 'g3': 'role2'}, {'g2': 'role1', 'g3': 'role2'}]}))

(b0721 and b0722 and b0724 and b0723)
(g1 and g3) or (g2 and g3)


In [ ]:
mrp0 = {
    'complex_ref': '~/template/complexes/name/sdh',
    'modelReactionProteinSubunits': [
        {'feature_refs': ['~/genome/features/id/b0721'], 'note': '', 'optionalSubunit': 0, 'role': 'sdhC', 'triggering': 1},
        {'feature_refs': ['~/genome/features/id/b0722'], 'note': '', 'optionalSubunit': 0, 'role': 'sdhD', 'triggering': 1},
        {'feature_refs': ['~/genome/features/id/b0723'], 'note': '', 'optionalSubunit': 0, 'role': 'sdhA', 'triggering': 1},
        {'feature_refs': ['~/genome/features/id/b0724'], 'note': '', 'optionalSubunit': 0, 'role': 'sdhB', 'triggering': 1}
    ],
   'note': '',
   'source': ''}
mrp1 = {
    'complex_ref': '~/template/complexes/name/pyk',
    'modelReactionProteinSubunits': [
        {'feature_refs': ['~/genome/features/id/b1676'], 'note': '', 'optionalSubunit': 0, 'role': 'pykAF', 'triggering': 1}
    ],
   'note': '',
   'source': ''}
mrp1 = {
    'complex_ref': '~/template/complexes/name/pyk',
    'modelReactionProteinSubunits': [
        {'feature_refs': ['~/genome/features/id/b1854'], 'note': '', 'optionalSubunit': 0, 'role': 'pykAF', 'triggering': 1}
    ],
   'note': '',
   'source': ''}
mrps = [
    mrp0
]
mrps

In [64]:
cpx

{'complexroles': [{'optional_role': 0,
   'templaterole_ref': '~/roles/id/ftr01381',
   'triggering': 1}],
 'confidence': 1,
 'id': 'cpx01814',
 'name': 'mscpx.1814',
 'reference': 'null',
 'source': 'ModelSEED'}

In [54]:
?template_reaction

Type:            NewModelTemplateReaction
String form:     rxn00154_c: cpd00003_c [c] + cpd00010_c [c] + cpd00020_c [c] --> cpd00004_c [c] + cpd00011_c [c] + cpd00022_c [c]
Length:          16
File:            /usr/local/lib/python3.8/site-packages/cobrakbase/core/kbasefba/new_template_reaction.py
Docstring:       <no docstring>
Class docstring: Base object to use for subobjects in KBase objects


In [51]:
template_reaction.get_complex_roles()

{'cpx01664': {'ftr00229'}}

In [47]:
rxn_roles['rxn00148_c']

set()

In [52]:
def aux_template(template):
    rxn_roles = {}
    roles = dict(map(lambda x: (x['id'], x), template.roles))
    for r in template.reactions:
        rxn_roles[r.id] = set()
        complex_roles = r.get_complex_roles()
        #print(complex_roles)
        if len(complex_roles) > 0:
            for cpx_id in complex_roles:
                for role_id in complex_roles[cpx_id]:
                    rxn_roles[r.id].add(normalize_role(roles[role_id]['name']))
                    # print(role_id, normalize_role(roles[role_id]['name']))
    return rxn_roles
rxn_roles = aux_template(template)

In [68]:
metabolic_reactions = {}
genome_search_names = set(search_name_to_genes)
for rxn_id in rxn_roles:
    sn_set = set(genome_search_names & rxn_roles[rxn_id])
    if len(sn_set) > 0:
        genes = set()
        # print(rxn_id, sn_set)
        for sn in sn_set:
            if sn in search_name_to_genes:
                genes |= search_name_to_genes[sn]
        metabolic_reactions[rxn_id] = genes

# temporary hack until proper complexes from template
metabolic_reactions_2 = {}
cpx_random = 0
for rxn_id in metabolic_reactions:
    metabolic_reactions_2[rxn_id] = {}
    for gene_id in metabolic_reactions[rxn_id]:
        metabolic_reactions_2[rxn_id]['complex' + str(cpx_random)] = {gene_id}
        cpx_random += 1

In [315]:
model_modelseed.reactions.rxn00001_c0.gene_reaction_rule = '(b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b3231 and b3314 and b0026 and b4129 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b3936 and b4203 and b2606 and b0023 and b1716 and b3299 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b3231 and b3314 and b0026 and b4129 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b0296 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b4203 and b2606 and b0023 and b1716 and b3299 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b4506 and b3231 and b3314 and b0026 and b4129 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b3936 and b4203 and b2606 and b0023 and b1716 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b4506 and b3231 and b3314 and b0026 and b4129 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b0296 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b4203 and b2606 and b0023 and b1716 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b3231 and b3314 and b0026 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b2890 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b3936 and b4203 and b2606 and b0023 and b1716 and b3299 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b3231 and b3314 and b0026 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b0296 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b2890 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b4203 and b2606 and b0023 and b1716 and b3299 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b4506 and b3231 and b3314 and b0026 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b2890 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b3936 and b4203 and b2606 and b0023 and b1716 and b3319 and b3341 and b3298) or (b3303 and b4202 and b3384 and b3302 and b0526 and b0169 and b3230 and b3320 and b3637 and b3297 and b3316 and b4200 and b3305 and b1866 and b2400 and b1876 and b3306 and b3309 and b3288 and b0893 and b2514 and b3560 and b1717 and b4506 and b3231 and b3314 and b0026 and b3307 and b0930 and b3186 and b0642 and b3321 and b2609 and b0296 and b2697 and b3312 and b1089 and b3301 and b3636 and b1713 and b3317 and b3559 and b3304 and b3984 and b3296 and b3310 and b3315 and b3294 and b3703 and b2890 and b3185 and b1719 and b3313 and b3311 and b1637 and b3318 and b3985 and b3165 and b1714 and b2114 and b3308 and b3342 and b3065 and b4258 and b4203 and b2606 and b0023 and b1716 and b3319 and b3341 and b3298) or (b0188) or (b2185) or (b3983) or (b3986)'

In [317]:
model_modelseed.reactions.rxn00001_c0.genes

frozenset({<Gene b0023 at 0x7f7e463f11f0>,
           <Gene b0026 at 0x7f7e463ecd90>,
           <Gene b0169 at 0x7f7e463ec400>,
           <Gene b0188 at 0x7f7e463ec8e0>,
           <Gene b0296 at 0x7f7e463ece20>,
           <Gene b0526 at 0x7f7e463ecc10>,
           <Gene b0642 at 0x7f7e463ec790>,
           <Gene b0893 at 0x7f7e463ec5b0>,
           <Gene b0930 at 0x7f7e463ec760>,
           <Gene b1089 at 0x7f7e463ec850>,
           <Gene b1637 at 0x7f7e463f1100>,
           <Gene b1713 at 0x7f7e463ece50>,
           <Gene b1714 at 0x7f7e463eca00>,
           <Gene b1716 at 0x7f7e463f1250>,
           <Gene b1717 at 0x7f7e463ec610>,
           <Gene b1719 at 0x7f7e463f10d0>,
           <Gene b1866 at 0x7f7e463ec550>,
           <Gene b1876 at 0x7f7e463eccd0>,
           <Gene b2114 at 0x7f7e463f1160>,
           <Gene b2185 at 0x7f7e463f1130>,
           <Gene b2400 at 0x7f7e464b5880>,
           <Gene b2514 at 0x7f7e463ec5e0>,
           <Gene b2606 at 0x7f7e463ecac0>,
           